# 构建检索问答链

在 `C3 搭建数据库` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [14]:
import sys
from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
from langchain.embeddings.baidu_qianfan_endpoint import QianfanEmbeddingsEndpoint

_ = load_dotenv(find_dotenv())  #读取对应KEY
sys.path.append("../C3 搭建知识库")
embedding = QianfanEmbeddingsEndpoint()

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [20]:
# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

In [21]:
os.listdir(persist_directory)

['chroma.sqlite3', '160b7709-d3d4-4f36-a583-f7fbd2d1daca']

In [22]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：5


我们可以测试一下加载的向量数据库，使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

> ⚠️使用相似性搜索前，请确保你已安装了 OpenAI 开源的快速分词工具 tiktoken 包：`pip install tiktoken`

In [33]:
question = "什么是法国的首都"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")

[INFO] [06-26 20:12:21] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /embeddings/embedding-v1


检索到的内容数：3


打印一下检索到的内容

In [34]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 与基础语言模型不同，指令微调 LLM 通过专门的训练，可以更好地理解并遵循指令。举个例子，当询问“法国的首都是什么？”时，这类模型很可能直接回答“法国的首都是巴黎”。指令微调 LLM 的训练通常基于预训练语言模型，先在大规模文本数据上进行预训练，掌握语言的基本规律。在此基础上进行进一步的训练与微调（finetune），输入是指令，输出是对这些指令的正确回复。有时还会采用RLHF（reinforcement learning from human feedback，人类反馈强化学习）技术，根据人类对模型输出的反馈进一步增强模型遵循指令的能力。通过这种受控的训练过程。指令微调 LLM 可以生成对指令高度敏感、更安全可靠的输出，较少无关和损害性内容。因此。许多实际应用已经转向使用这类大语言模型。
-----------------------------------------------------
检索到的第1个内容: 
 随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前，有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如果你以“法国的首都是什么”为 Prompt ，则基础 LLM 可能会根据互联网上的文章，将回答预测为“法国最大的城市是什么？法国的人口是多少？”，因为互联网上的文章很可能是有关法国国家的问答题目列表。
-----------------------------------------------------
检索到的第2个内容: 
 第一章 简介

欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Large 

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [19]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

llm.invoke("请你自我介绍一下自己！")

AIMessage(content='你好，我是一个智能助手，专门为用户提供各种服务和帮助。我可以回答问题、提供信息、解决问题等等。如果您有任何需要，请随时告诉我，我会尽力帮助您的。感谢您的使用！', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 20, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [35]:
from dotenv import find_dotenv, load_dotenv
import os
from langchain_community.llms import QianfanLLMEndpoint

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
QIANFAN_AK = os.environ["QIANFAN_AK"]
QIANFAN_SK = os.environ["QIANFAN_SK"]
llm = QianfanLLMEndpoint(streaming=True)
res = llm("你好，请你自我介绍一下！")
print(res)

[WARNING] [06-26 20:14:38] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:14:38] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant


你好！我是文心一言，英文名是ERNIE Bot。我是一款人工智能语言模型，可以协助你完成范围广泛的任务并提供有关各种主题的信息，比如回答问题，提供定义和解释及建议，如果你有任何问题，请随时向我提问。


## 3. 构建检索问答链

In [36]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


再创建一个基于模板的检索链：

In [37]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


创建检索 QA 链的方法 RetrievalQA.from_chain_type() 有如下参数：
- llm：指定使用的 LLM
- 指定 chain type : RetrievalQA.from_chain_type(chain_type="map_reduce")，也可以利用load_qa_chain()方法指定chain type。
- 自定义 prompt ：通过在RetrievalQA.from_chain_type()方法中，指定chain_type_kwargs参数，而该参数：chain_type_kwargs = {"prompt": PROMPT}
- 返回源文档：通过RetrievalQA.from_chain_type()方法中指定：return_source_documents=True参数；也可以使用RetrievalQAWithSourceChain()方法，返回源文档的引用（坐标或者叫主键、索引）

## 4.检索问答链效果测试

In [38]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [39]:
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

[INFO] [06-26 20:16:08] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-26 20:16:09] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:16:09] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant


大模型+知识库后回答 question_1 的结果：
南瓜书是指《Prompt Engineering for Developer》课程，是由吴恩达老师与OpenAI技术团队成员Isa Fulford老师合作编写的，主要面向开发人员的提示词设计的教材。


In [40]:
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

[INFO] [06-26 20:16:16] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-26 20:16:16] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:16:16] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant


大模型+知识库后回答 question_2 的结果：
王阳明是一位中国明代哲学家，被誉为“心学之宗”。


### 4.2 大模型自己回答的效果

In [41]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm.predict(prompt_template)

[WARNING] [06-26 20:16:36] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:16:36] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant


'南瓜书是一种儿童读物，通常由南瓜制成，里面填充着故事书、玩具、糖果等物品。南瓜书既有阅读的功能，也有玩具和装饰的作用，受到许多家庭的喜爱。\n\n除了南瓜书之外，其他一些儿童读物也可以称为南瓜书，如苹果书、樱桃书等。这些儿童读物通常具有趣味性和实用性，可以激发孩子的阅读兴趣和想象力。同时，它们也是许多家庭装饰品的一种，为家庭增添了温馨和乐趣。'

In [42]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm.predict(prompt_template)

[WARNING] [06-26 20:16:40] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:16:40] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant


'王阳明是明朝时期的一位著名思想家、哲学家，也是心学理论的创始人。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


## 1. 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [43]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

## 2. 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

![](../../figures/Modular_components.png)

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习到关于提示工程的知识吗？”，并查看回答。

In [44]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa({"question": question})
print(result['answer'])

[INFO] [06-26 20:31:39] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-26 20:31:40] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:31:40] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant


是的，你可以学习到关于提示工程的知识。

在上述文本中，提到了一个面向开发者的提示工程部分，这是由吴恩达老师的《Prompt Engineering for Developer》课程编写而成的。这个课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并在教授 LLM （大语言模型）技术方面做出了很大贡献。通过本模块的学习，与读者分享提升大语言模型应用效果的各种技巧和最佳实践。这些知识包括软件开发提示词设计、文本总结、推理、转换、扩展以及构建聊天机器人等语言模型典型应用场景。网络上也有许多关于提示词设计的材料，例如《30 prompts everyone has to know》之类的文章，但是这类的知识通常基于特定的大语言模型应用如 ChatGPT 的 Web 界面，专门适用于一次性的任务。但对于开发人员来说，提示工程的关键是了解如何通过 API 接口调用 LLM API 来进行更加灵活的应用程序构建。文中也提到，DeepLearning.AI 的姊妹公司 AI Fund 的团队一直在与许多初创公司合作，将这些技术应用于诸多应用程序上。因此，对于想要学习提示工程知识的开发者来说，这是一个非常有用的课程和资源。


然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [45]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])

[WARNING] [06-26 20:31:57] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:31:57] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant
[INFO] [06-26 20:31:57] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [06-26 20:31:58] base.py:516 [t:140714638276096]: This key `stop` does not seem to be a parameter that the model `ERNIE-Bot-turbo` will accept
[INFO] [06-26 20:31:58] openapi_requestor.py:316 [t:140714638276096]: requesting llm api endpoint: /chat/eb-instant


这门课需要向开发者传授提示工程方面的知识，是因为提示词的设计对于提升大语言模型的应用效果非常重要。通过学习如何设计有效的提示词，开发者可以更好地利用大语言模型API快速构建软件应用程序，从而提升应用程序的性能和用户体验。此外，通过指令微调训练大语言模型，可以生成对指令高度敏感、更安全可靠的输出，减少无关和损害性内容，这对于许多实际应用来说是非常有帮助的。因此，这门课旨在帮助开发者掌握提示工程方面的知识和技巧，以便更好地开发和利用大语言模型的应用。


可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。